This is a [jupyter](https://jupyter.org/) notebook that contains text, equations, images and executable code in one document.

$
\newcommand{\dd}{\text{d}}
\newcommand{\e}{\operatorname{e}}
\newcommand{\Fc}{\underline F}
\newcommand{\Hc}{\underline H}
\newcommand{\Xc}{\underline X}
\newcommand{\Yc}{\underline Y}
\newcommand{\Zc}{\underline Z}
\newcommand{\xic}{\underline \xi}
\newcommand{\vc}{\underline v}
\newcommand{\pc}{\underline p}
\newcommand{\Ec}{\underline E}
\newcommand{\kc}{\underline k}
\newcommand{\xiv}{\pmb{\xi}}
\newcommand{\Psiv}{\pmb{\Psi}}
\newcommand{\xivc}{\underline \xiv}
\newcommand{\xv}{\mathbf{x}}
\newcommand{\jj}{\operatorname{j}}
$

# 11 Finite Structures: Plates, Energy Analysis

* we know about the sound field in finite beams
* derivation was not exactly specific to beams and can be repeated for plates

## 11.1 Bending Waves in Finite Plates
* we assume simple homogeneous plate area density $m''$ and bending stiffness $B'$
* possible (simple) boundary conditions:
  * free
  * clamped
  * simply supported
  * (guided)
* geometric parameters depend on plate shape (rectangular, circular, triangular, arbitrary)
* no closed solution for arbitrary shape and boundary conditions
* hypothesis: boundary conditions do only matter for the first few modes (compare to beams)
* hypothesis: shape has a big influence on mode shape, but less so on eigenfrequencies (except for the first few)

#### Simply supported, rectangular plate
![](tikzfig/finite_plate.svg)
* shape: rectangle with side lengths $l_x, l_z$
* boundary conditions:
  * $x=0,x=l_x$: $M_z'=0, \xi_y=0$
  * $z=0,z=l_z$: $M_x'=0, \xi_y=0$
* mode shapes are (compare to simply supported beam)
\begin{gather}\label{eq:mode_shape}
\varphi_N(x)=\sin (k_{B,N_1}x)\,\sin (k_{B,N_2}z)\\
k_{B,N_1}=\frac{\pi N_1}{l_x}, \; k_{B,N_2}=\frac{\pi N_2}{l_z}
\end{gather}

In [1]:
%matplotlib inline
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.pyplot as pl
import ipywidgets as ipw

@ipw.interact(t=(0,10,0.2),N1=(1,5,1),N2=(1,5,1))
def plot_modes_plate(t=0,N1=1,N2=1):
    # assume lx=4, lz=3
    x, z = np.meshgrid(np.linspace(0,4,21), np.linspace(0,3,16))
    # assume amplitudes=0.1, l=10, omega=1
    xi = np.sin(np.pi*N1*x/4)*np.sin(np.pi*N2*z/3)*np.cos(t)
    fig = pl.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(x,z,0.2*xi,cmap='coolwarm',vmin=-0.2,vmax=0.2)
    ax.set_zlim(-1.01, 1.01)

interactive(children=(FloatSlider(value=0.0, description='t', max=10.0, step=0.2), IntSlider(value=1, descript…

* like in the case of the beam the free bending wave number matches the two wavenumbers together if the frequency is an eigenfrequency
\begin{equation}
k_B^2 = k_{B,N_1}^2 + k_{B,N_2}^2
\end{equation}
\begin{equation}
k_B^4=\frac{m''\omega^2}{B'}  
\end{equation}
* calculate the eigenfrequencies:

In [2]:
import sympy as sy
omN, kB, kBN1, kBN2 = sy.symbols("omega_N k_B k_BN1 k_BN2", positive=True)
B1, m2, lx, lz  = sy.symbols("B' m'' l_x l_z", positive=True)
N1, N2 = sy.symbols("N1 N2",integer=True,positive=True)
kBN1 = N1*sy.pi/lx
kBN2 = N2*sy.pi/lz
kB = sy.root(m2/B1,4)*sy.sqrt(omN)
omN_=sy.solve(sy.Eq(kB**2,kBN1**2+kBN2**2),omN)[0].expand().collect(B1/m2)
display(sy.Eq(omN,omN_))

Eq(omega_N, pi**2*sqrt(B')*N1**2/(l_x**2*sqrt(m'')) + pi**2*sqrt(B')*N2**2/(l_z**2*sqrt(m'')))

* each pair $(N_1,N_2)$ corresponds to one eigenfrequency
* multiplicity: different pairs $(N_1,N_2)$ may correspond to the same eigenfrequency
* example for a plate with $l_x : l_z = 4:2$:

In [3]:
import pandas as pd
N1_ = np.arange(1,9)[np.newaxis]
N2_ = np.arange(1,8)[:,np.newaxis]
# assume lx:lz = 4:2 and sqrt(B1/m2)=1
lx_, lz_ = 4, 2
omN_ = np.pi**2 * ((N1_/lx_)**2 + (N2_/lz_)**2)
# use pandas dataframe for convenient formatting
pd.DataFrame(omN_, columns=N1_[0], index= N2_[:,0])

,1,2,3,4,5,6,7,8
1,3.084251,4.934802,8.019054,12.337006,17.888658,24.674011,32.693065,41.945819
2,10.486455,12.337006,15.421257,19.739209,25.290861,32.076214,40.095268,49.348022
3,22.823460,24.674011,27.758262,32.076214,37.627867,44.413220,52.432273,61.685028
4,40.095268,41.945819,45.030070,49.348022,54.899674,61.685028,69.704081,78.956835
5,62.301878,64.152429,67.236680,71.554632,77.106284,83.891637,91.910691,101.163445
6,89.443290,91.293841,94.378092,98.696044,104.247696,111.033050,119.052103,128.304857
7,121.519504,123.370055,126.454306,130.772258,136.323911,143.109264,151.128317,160.381072


* some eigenfrequencies appear twice, e.g. for (1,4) and (2,2)

## 11.2 Excitation of Bending Waves
#### Mode summation for simply supported rectangular plate
* inhomogeneous wave equation
\begin{equation}\label{eq:iwaveeq}
\left(\frac{\partial^2}{\partial x^2}+\frac{\partial^2}{\partial z^2}\right)^2\xic_y(x,z)+k_B^4 \xic_y(x,z)=\frac{\pc(x,z)}{B'}
\end{equation}
* plate is excited by spatial distribution of pressure $\pc$ on surface
* point excitation:
\begin{equation}
\pc(x,z)=\Fc_0\delta(x-x_1)\delta(z-z_1)
\end{equation}
* mode summation (compare to derivation for beam)
\begin{equation}
\xic_y(x,z)=\sum_{N=1}^{\infty} \underline C_N \varphi_N(x,z)=\sum_{N_1=1}^{\infty}\sum_{N_2=1}^{\infty} \underline C_N \varphi_N(x,z)
\end{equation}
* single summation becomes double summation, where factor $\underline C_N$ and the mode shape \eqref{eq:mode_shape} depend on both $N_1$ and $N_2$
* mode summation coefficients:
\begin{equation}
\underline C_N=\frac{1}{B'\left(\left(k_{B,N_1}^2+k_{B,N_2}^2\right)^2-k_{B}^4\right)}
\frac{
\int_0^{l_x}\int_0^{l_z}\pc(x,z)\varphi_N(x,z) \dd z \dd x
}{
\int_0^{l_x}\int_0^{l_z}\varphi_N^2(x,z) \dd z \dd x
}
\end{equation}

* norm of mode shape 

In [4]:
x, z = sy.symbols("x z", real=True)
sy.integrate((sy.sin(sy.pi*N1*x/lx)*sy.sin(sy.pi*N2*z/lz))**2,
             (x,0,lx),(z,0,lz))

l_x*l_z/4

![](tikzfig/finite_plate_point_excitation.svg)
* mode summation coefficients for point excitation (wave numbers substituted with frequencies, complex bending stiffness $\underline B' = B'(1+\jj \eta)$)
\begin{equation}\label{eq:CN}
\underline C_N=\frac{4 \Fc_0\varphi_N(x_1,z_1)}
{m'' l_x l_z\left(\omega_{N}^2 \left(1+\jj \eta\right)-\omega^2\right)}
\end{equation}
* wave field for point excitation 
\begin{multline}
\xic_y(x,z)=\\
\frac{4\Fc_0}{m'' l_x l_z}
\sum_{N_1=1}^{\infty}\sum_{N_2=1}^{\infty} 
\frac{\sin k_{B,N_1}x \sin k_{B,N_2}z \sin k_{B,N_1}x_1 \sin k_{B,N_2}z_1}
{\omega_{N}^2 \left(1+\jj \eta\right)-\omega^2}
\end{multline}

In [5]:
@ipw.interact(tom=(0,10,0.1),om=(0.5,15,0.5),
              x1=(0,4,0.25),z1=(0,3,0.25),eta=(0.01,0.1,0.01))
def plot_modesum_plate(tom=0,om=1.5,x1=2.5,z1=1.0,eta=0.01):
    # assume lx=4, lz=3
    lx, lz = 4, 3
    # get coordinates for plot, prepare two extra dims for summation
    x, z = np.meshgrid(np.linspace(0,4,17), np.linspace(0,3,13))
    x = x[:,:,np.newaxis,np.newaxis]
    z = z[:,:,np.newaxis,np.newaxis]
    # sum over N1=1...6, N2=1...5 instead of to infinity
    # first two dims are for x,z
    N1 = np.arange(1,6).reshape((1,1,-1,1))
    N2 = np.arange(1,5).reshape((1,1,1,-1))
    # assume B1=1 m2=1
    # compute each element of the sum in one go
    omN = np.pi**2 * ((N1/lx)**2 + (N2/lz)**2)
    xi = 4/lx/lz *\
        np.sin(np.pi*N1*x/lx)*np.sin(np.pi*N1*x1/lx)* \
        np.sin(np.pi*N2*z/lz)*np.sin(np.pi*N2*z1/lz)/ \
        (omN**2 * (1+1j*eta)-om**2)
    # double sum 
    xi = xi.sum(axis=(2,3))
    vmax = np.abs(xi).max()
    # add time dependency
    xi = xi*np.exp(1j*tom)
    # plot 3D
    fig = pl.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(x[:,:,0,0],z[:,:,0,0],xi.real,cmap='coolwarm',
                   vmin=-vmax-0.01,vmax=vmax+0.01)
    ax.set_zlim(-2*vmax, 2*vmax)

interactive(children=(FloatSlider(value=0.0, description='tom', max=10.0), FloatSlider(value=1.5, description=…

## 11.3 Point Impedances

#### Simply supported rectangular plate
* displacement at excitation point:
\begin{multline}
\xic_y(x_1,z_1)=
\frac{4 \Fc_0}{m'' l_x l_z}
\sum_{N_1=1}^{\infty}\sum_{N_2=1}^{\infty} 
\frac{\sin^2 k_{B,N_1}x_1 \sin^2 k_{B,N_2}z_1}
{\omega_{N}^2 \left(1+\jj \eta\right)-\omega^2}
\end{multline}
* point impedance
\begin{equation}\label{eq:}
\Zc(x_1,z_1)=\frac{\Fc_0}{\vc_y(x_1,z_1)}=\frac{\Fc_0}{\jj \omega \xic_y(x_1,z_1)}
\end{equation}
* plot this and compare to infinite plate

In [6]:
@ipw.interact(x1=(0.25,3.75,0.25),z1=(0.25,2.75,0.25),eta=(0.01,0.1,0.01))
def plot_plate_impedance(x1=2.5,z1=1.0,eta=0.01):
    # assume lx=4, lz=3
    lx, lz = 4, 3
    om = np.linspace(0.5,15,200).reshape((-1,1,1))
    # sum over N1=1...6, N2=1...5 instead of to infinity
    N1 = np.arange(1,6).reshape((1,-1,1))
    N2 = np.arange(1,5).reshape((1,1,-1))
    # assume sqrt(B1/m2)=1
    # compute each element of the sum in one go
    omN = np.pi**2 * ((N1/lx)**2 + (N2/lz)**2)
    xi = 4/lx/lz*\
        np.sin(np.pi*N1*x1/lx)**2 * np.sin(np.pi*N2*z1/lz)**2/ \
        (omN**2 * (1+1j*eta)-om**2)
    # double sum 
    xi = xi.sum(axis=(1,2))
    Z = 1./(1j*om[:,0,0]*xi)
    Zinf = 8*om[:,0,0]/om[:,0,0]
    # plot
    pl.semilogy(om[:,0,0],np.abs(Z),label=r'$|Z|$')
    pl.semilogy(om[:,0,0],Zinf,label=r'${Z_\infty}$')
    omN1 = omN.flatten()
    omN1 = omN1[omN1<15]
    pl.plot(omN1,np.ones_like(omN1),'o',label=r'$\omega_N$')
    pl.xlabel(r'$\omega$')
    pl.ylabel(r'$|Z|$')
    pl.legend()

interactive(children=(FloatSlider(value=2.5, description='x1', max=3.75, min=0.25, step=0.25), FloatSlider(val…

* the impedance is not too different from that of an infinite homogeneous plate
* for other shapes and boundary conditions the impedance can hardly be computed the same way (one would have to use numerical techniques like Finite Element Method)
* alternative: statistical approach - compute "mean value" using power and energy analysis

## 10.4 Energy and Power Input

#### Energy

* (kinetic) energy
\begin{align}
W&=W_{kin}+W_{pot}=2W_{kin}=2 \int w_{kin} \dd S\\
&= \iint \frac{m''}{2} |\vc(x,z)|^2 \dd x \dd z
\end{align}
* if the plate is homogeneous, the $m''$ is constant
\begin{align}
W&= \frac{m}{2} \frac{1}{S} \iint |\vc(x,z)|^2 \dd x \dd z\\
&= \frac{m}{2} \overline{v^2}
\end{align}
* mode summation can be applied 
\begin{align}
\overline{v^2}&= \frac{1}{S}\iint |\vc(x,z)|^2 \dd x \dd z\\
&=  \frac{\omega^2}{S}\iint \left|\sum_{N_1}^\infty \sum_{N_2}^\infty 
\underline C_N \varphi_N(x,z)
\right|^2 \dd x \dd z\\
&=  \frac{\omega^2}{S}\sum_{N_1}^\infty \sum_{N_2}^\infty 
\left| \underline C_N \right|^2
\iint  \varphi_N(x,z)^2\dd x \dd z
\end{align}
where the last substitution make use of the orthogonality property of $\varphi(x,z)$
* integral is norm of mode shape $\frac{l_x l_z}{4}$
* we get the mode coefficients from \eqref{eq:CN} and after substitution
\begin{align}
\overline{v^2}&=  \frac{4\omega^2|\Fc_0|^2}{S^2 m''^2}\sum_{N_1}^\infty \sum_{N_2}^\infty 
\frac{\varphi_N^2(x_1,z_1)}
{\left|\omega_{N}^2 \left(1+\jj \eta\right)-\omega^2\right|^2} \\
&=\frac{4\omega^2|\Fc_0|^2}{S^2 m''^2}\sum_{N_1}^\infty \sum_{N_2}^\infty
\frac{\varphi_N^2(x_1,z_1)}
{\left(\omega_{N}^2 -\omega^2\right)^2+\eta^2\omega_N^4}
\end{align}
* plot this for different excitation points

In [7]:
@ipw.interact(x1=(0.25,3.75,0.25),z1=(0.25,2.75,0.25),eta=(0.01,0.1,0.01))
def plot_v2(x1=2.5,z1=1.0,eta=0.01):
    # assume lx=4, lz=3
    lx, lz = 4, 3
    om = np.linspace(0.5,15,100).reshape((-1,1,1))
    # sum over N1=1...6, N2=1...5 instead of to infinity
    N1 = np.arange(1,6).reshape((1,-1,1))
    N2 = np.arange(1,5).reshape((1,1,-1))
    # assume B1=1 m2=1 F0=1
    # compute each element of the sum in one go
    omN = np.pi**2 * ((N1/lx)**2 + (N2/lz)**2)
    v2 = 4*om**2/lx**2/lz**2 *\
        np.sin(np.pi*N1*x1/lx)**2 * np.sin(np.pi*N2*z1/lz)**2/ \
        ((omN**2-om**2)**2+eta**2 * omN**4)
    # double sum 
    v2 = v2.sum(axis=(1,2))
    # plot
    pl.semilogy(om[:,0,0],v2,label=r'$v^2$')
    pl.ylim(1e-3,1e1)
    pl.xlabel(r'$\omega$')
    pl.ylabel(r'$v^2$')

interactive(children=(FloatSlider(value=2.5, description='x1', max=3.75, min=0.25, step=0.25), FloatSlider(val…

* looks always similar - same peaks, different heights
* take a second average, now over the excitation point:
\begin{align}
\overline{\overline{v^2}} &=\frac{1}{S}\iint \overline{v^2} \dd x_1 \dd z_1\\
&=\frac{\omega^2|\Fc_0|^2}{S^2 m''^2}\sum_{N_1}^\infty \sum_{N_2}^\infty
\frac{1}
{\left(\omega_{N}^2 -\omega^2\right)^2+\eta^2\omega_N^4}
\end{align}


In [8]:
@ipw.interact(eta=(0.01,0.1,0.01))
def plot_v2(eta=0.01):
    # assume lx=4, lz=3
    lx, lz = 4, 3
    om = np.linspace(0.5,15,100).reshape((-1,1,1))
    # sum over N1=1...6, N2=1...5 instead of to infinity
    N1 = np.arange(1,6).reshape((1,-1,1))
    N2 = np.arange(1,5).reshape((1,1,-1))
    # assume B1=1 m2=1 F0=1
    # compute each element of the sum in one go
    omN = np.pi**2 * ((N1/lx)**2 + (N2/lz)**2)
    v2 = om**2/lx**2/lz**2/((omN**2-om**2)**2+eta**2 * omN**4)
    # double sum 
    v2 = v2.sum(axis=(1,2))
    # plot
    pl.semilogy(om[:,0,0],v2,label=r'$v^2$')
    pl.ylim(1e-3,1e1)
    pl.xlabel(r'$\omega$')
    pl.ylabel(r'$v^2$')

interactive(children=(FloatSlider(value=0.01, description='eta', max=0.1, min=0.01, step=0.01), Output()), _do…

* often, we are not interested in the response at a certain frequency $\omega$, but in the average response within a certain frequency band $\omega_a < \omega < \omega_b$
* octave band: $\omega_b = 2\omega_a$
* third octave band: $\omega_b = \sqrt[3]{2}\omega_a$
* third average 
\begin{align}
\overline{\overline{\overline{v^2}}} &=\frac{1}{\omega_b-\omega_a} \int_{\omega_a}^{\omega_b} \overline{\overline{v^2}} \dd \omega\\
&=\frac{|\Fc_0|^2}{S^2 m''^2}\sum_{N_1}^\infty \sum_{N_2}^\infty
\frac{1}{\omega_b-\omega_a} \int_{\omega_a}^{\omega_b}
\frac{\omega^2}
{\left(\omega_{N}^2 -\omega^2\right)^2+\eta^2\omega_N^4} \dd \omega
\end{align}
where we assume that $\Fc_0$ does not depend on $\omega$ ("white" power spectrum)
* under the assumption of $\eta\ll 1$ the integral can be approximated by
\begin{equation}
\int_{\omega_a}^{\omega_b} \dots \dd \omega \approx
\begin{cases}
\frac{\pi}{2\eta\omega_N}& \omega_a < \omega < \omega_b\\
0 & \text{otherwise}
\end{cases}
\end{equation}
* the average is now:
\begin{align}
\overline{\overline{\overline{v^2}}} &\approx\frac{|\Fc_0|^2}{S^2 m''^2}
\frac{1}{\omega_b-\omega_a} \sum_{N=N_a}^{N_b}\frac{\pi}{2\eta\omega_N}
\end{align}
where $N_a$ and $N_b$ are the index of the lowest and highest eigenfrequency in the band, respectively
* if we now assume that the remaining $\omega_N$ are not too different and can be replaced by a average frequency $\omega=\sqrt{\omega_a \omega_b}$ (which is equivalent to the assumption of equipartition of energy over the modes), then this simplifies to
\begin{align}\label{eq:mean_velo}
\overline{\overline{\overline{v^2}}} &\approx\frac{|\Fc_0|^2 \pi}{S^2 m''^2 2\eta\omega}
\frac{N_b-N_a+1}{\omega_b-\omega_a} 
\end{align}
* average squared velocity depends on modal density (modes per band)
\begin{equation}
n = \frac{N_b-N_a+1}{\omega_b-\omega_a}=\frac{\Delta N}{\Delta \omega}
\end{equation}

#### Power input
* power input (compare to discussion of infinite plate)
\begin{equation}
P = \frac{1}{2}\Re\left(\iint\pc(x,z)\vc^*(x,z)\dd x \dd z\right)
\end{equation}
* point excitation, mode summation, and average over excitation points and frequency band gives
\begin{equation}\label{eq:mean_power}
P=\frac{|\Fc_0|^2}{2}\frac{\pi}{2m''S}n
\end{equation}
* on the other hand the power input is related to the point impedance (see prior discussion of impedance) 
\begin{equation}
P=\frac{|\Fc_0|^2}{2}\Re\left(\frac{1}{\Zc}\right)
\end{equation}
* interesting result: impedance and modal density are related!
\begin{equation}
\Re\left(\frac{1}{\Zc}\right)=\frac{\pi}{2m''S}n
\end{equation}
* example: the impedance for the infinite plate was found to be $Z=8\sqrt{B'm''}$, then
\begin{equation}
n= \frac{2m''S}{\pi 8\sqrt{B'm''}}=\frac{S}{4\pi}\sqrt{\frac{m''}{B'}}
\end{equation}
* does _not_ depend on frequency
* this result can also be found as expected value by counting the eigenfrequncies per band
* moreover, comparing \eqref{eq:mean_power} and \eqref{eq:mean_velo} gives
\begin{align}
P &= \frac{\omega\eta}{2}Sm'' \overline{\overline{\overline{v^2}}}\\
  &= \omega\eta \overline W
\end{align}
* given the mean velocity or expected value of energy per frequency band, the power input can be computed
* this is equivalent to _diffuse_ wave field assumption
* basis of __statistical energy analysis__ (next topic)


#### ----------------------------------------
* wave field in finite plates can also be described in terms of modes
* multiple eigenfrequencies may occur
* wave field for arbitrary excitation by mode summation technique 
* point impedance from mode summation, similar to infinite plate
* energetic average over plate, excitation location and frequency band
* modal density is related to point impedance
* input power per frequency band is statistically related to energy 

#### License

This notebook is an [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use it for your own purposes. The text and the images are licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), and any code under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: Ennes Sarradj, Structure-borne sound lecture notes, 2019.

In [ ]:
# this is just for custom formatting
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()